In [1]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import glob
import os
import pandas as pd
from tensorflow.keras import datasets, layers, models
import efficientnet.tfkeras as efn

In [2]:
# 文件存放位置
# data3>train>0
# data3>train>1
# data3>train>2
# data3>train>3
# data3>test>测试集

In [ ]:
df = pd.read_csv('train.csv')
df = df.set_index('image')

In [3]:
train_image_path = glob.glob('.\\train\\*.jpg')
labels = [df.loc[i.split('\\')[-1]][0]-1 for i in train_image_path]

In [7]:
# Step1 使用 tf.data.Dataset.from_tensor_slices 进行加载
image_ds = tf.data.Dataset.from_tensor_slices((train_image_path,labels))

In [8]:
val_count = int(len(labels)*0.2)
train_count = len(labels)-val_count

In [9]:
def load_preprocess_image(path,label):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image,channels=3)
    image = tf.image.resize(image,[300,300])
    image = tf.image.random_crop(image,[260,260,3])
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_brightness(image,0.2)
    
    image = tf.cast(image,tf.float32)
    image = image/255
    label = tf.reshape(label,[1])
    return image,label

In [10]:
def load_preprocess_image_test(path,label):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image,channels=3)
    image = tf.image.resize(image,[260,260])
    image = tf.cast(image,tf.float32)
    image = image/255
    label = tf.reshape(label,[1])
    return image,label

In [11]:
# 选取验证集
image_train_ds = image_ds.skip(val_count)
image_val_ds = image_ds.take(val_count)

# Step3 预处理 (预处理函数在下面)
AUTOTUNE = tf.data.experimental.AUTOTUNE
image_train_ds = image_train_ds.map(load_preprocess_image,num_parallel_calls=AUTOTUNE)
image_val_ds = image_val_ds.map(load_preprocess_image_test,num_parallel_calls=AUTOTUNE)

In [12]:
BATCH_SIZE = 32
# shuffle 打乱数据 batch设置 batch size repeat设置迭代次数(迭代2次) test数据集不需要
image_train_ds = image_train_ds.repeat(20).shuffle(train_count).batch(BATCH_SIZE)
image_train_ds = image_train_ds.prefetch(AUTOTUNE)#预取,GPU，CPU加速

image_val_ds = image_val_ds.batch(BATCH_SIZE)
image_val_ds = image_val_ds.prefetch(AUTOTUNE)#预取

In [ ]:
model = keras.Sequential()
model.add(covn_base)
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(1024,activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(512,activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(512,activation='relu'))
model.add(keras.layers.Dense(8,activation='sigmoid'))

covn_base.trainable = False #设置权重参数不可训练

In [17]:
#编译
model.compile(optimizer=keras.optimizers.Adam(lr=0.001),
             loss = 'sparse_categorical_crossentropy',
             metrics=['acc'])

#训练
history = model.fit(
    image_train_ds,
    steps_per_epoch=train_count//BATCH_SIZE,
    epochs=8,
    validation_data=image_val_ds,
    validation_steps=val_count//BATCH_SIZE
)

Epoch 1/8
8/8 [==============================] - 71s 9s/step - loss: 1.9182 - acc: 0.2852 - val_loss: 1.9199 - val_acc: 0.3125
Epoch 2/8
8/8 [==============================] - 84s 11s/step - loss: 1.8865 - acc: 0.3633 - val_loss: 1.9110 - val_acc: 0.3125
Epoch 3/8
8/8 [==============================] - 72s 9s/step - loss: 1.8530 - acc: 0.3438 - val_loss: 1.8275 - val_acc: 0.3125
Epoch 4/8
8/8 [==============================] - 71s 9s/step - loss: 1.7756 - acc: 0.3164 - val_loss: 1.8526 - val_acc: 0.3125
Epoch 5/8
8/8 [==============================] - 70s 9s/step - loss: 1.6548 - acc: 0.3906 - val_loss: 1.8291 - val_acc: 0.3125
Epoch 6/8
8/8 [==============================] - 67s 8s/step - loss: 1.6167 - acc: 0.3281 - val_loss: 1.8404 - val_acc: 0.3281
Epoch 7/8
8/8 [==============================] - 67s 8s/step - loss: 1.5572 - acc: 0.3633 - val_loss: 1.7978 - val_acc: 0.3438
Epoch 8/8
3/8 [==========>...................] - ETA: 27s - loss: 1.5977 - acc: 0.3646

KeyboardInterrupt: 

In [33]:
covn_base.trainable=True

fine_tune_at = -3
for layer in covn_base.layers[:fine_tune_at]:
    layer.trainable = False

model.compile(optimizer=keras.optimizers.Adam(0.0005),
             loss = 'sparse_categorical_crossentropy',
             metrics=['acc'])

initial_epochs=8
fine_tune_epochs=8
total_epoch = initial_epochs+fine_tune_epochs

#二次训练
history = model.fit(
    image_train_ds,
    steps_per_epoch=train_count//BATCH_SIZE,
    epochs=total_epoch,
    initial_epoch=initial_epochs,
    validation_data=image_val_ds,
    validation_steps=val_count//BATCH_SIZE
)

model.save('mstz_model_EfficientNetB52.h5')

Epoch 9/16
8/8 [==============================] - 39s 5s/step - loss: 1.6745 - acc: 0.3242 - val_loss: 1.8625 - val_acc: 0.3125
Epoch 10/16
8/8 [==============================] - 36s 4s/step - loss: 1.6012 - acc: 0.3750 - val_loss: 1.8226 - val_acc: 0.3125
Epoch 11/16
1/8 [==>...........................] - 8s 8s/step - loss: 1.4450 - acc: 0.3000 - val_loss: 1.8243 - val_acc: 0.3125


In [34]:
#加载图片
def load_preprocess_images(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image,channels=3)
    image = tf.image.resize(image,[260,260])
    image = tf.cast(image,tf.float32)
    image = image/255
    image = tf.expand_dims(image,0)
    return image

In [96]:
test_image_path = glob.glob('test/testA/*.jpg')

images = [load_preprocess_images(image_path) for image_path in test_image_path]

image_count = len(images)
values = []
result_dict = {}

for i in range(image_count):
    pred = model.predict(images[i])
    values.append(np.argmax(pred))
#     if pred[0][0]>0.25:
#         values.append(1)
#     else:
#         values.append(np.argmax(pred[0][1:])+2)
    print('.',end='')

......................................................................................................................................................

In [99]:
#写文件
with open('sub.csv','w',encoding='utf-8') as f:
    f.write('image_id,category_id\n')
    [f.write('{0},{1}\n'.format(key.split('\\')[1][:-4]+'.jpg', value+1)) for (key,value) in zip(test_image_path,values)]